# 📝 Formatted Output

Getting your data back in a format that can be consistently and reliably used is a problem you will likely encounter. Especially when creating usable data to feed into an API. Imagine for example you are building a service to help authors write books. One feature you may provide is character creation. While keeping the human in the loop is good, we don't want our users needing to copy and paste from the LLM response or get inconsistent results.

Fortunately we  can solve that problem with formatted output!

## 🛠️ Setup

We will start by installing the packages we need and creating a connection to the LLM.
> *Note:* Because this feature is not yet released directly for the Ollama API, we will use a different connector. Ollama will still be providing the results.

In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.33.0"
#r "nuget: Microsoft.Extensions.AI.Ollama, 9.0.1-preview.1.24570.5"

using System.ComponentModel;
using System.Text.Json;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var ollamaApiUrl = Environment.GetEnvironmentVariable("DOTNET_RUNNING_IN_CONTAINER") == "true"
  ? "http://host.docker.internal:11434"
  : "http://localhost:11434";
var openAiFormattedUrl = $"{ollamaApiUrl}/v1";

// Setup the model we want to use
var languageModel = "llama3.2:3b";

// Create the kernel
#pragma warning disable SKEXP0010
var kernel = Kernel
  .CreateBuilder()
  .AddOpenAIChatCompletion(languageModel, new Uri(openAiFormattedUrl), "ollama")
  .Build();

## 👩🏿‍🎤 Define our character class

Now we need to create a class to hold our character data. We will use the `[Description]` attribute to ensure the model knows what our fields mean and can fill them in properly.

In [ ]:
public struct MyGreatNovelCharacter
{
    [Description("The character's name")]
    public string Name { get; set; }

    [Description("The character's background story")]
    public string BackgroundStory { get; set; }

    [Description("The name of the town the character lives in")]
    public string Town { get; set; }

    [Description("The character's age")]
    public int Age { get; set; }

    [Description("The job the character has in town")]
    public string Occupation { get; set; }

    [Description("The character's race")]
    public string Race { get; set; }

    [Description("Names of the character's friends")]
    public List<string> Friends { get; set; }

    [Description("The character's personality traits")]
    public List<string> PersonalityTraits { get; set; }
}

## 🤖 Generate a character!

We're all set to get a character from the model, so let's make the request.

In [ ]:
#pragma warning disable SKEXP0010

var result = await kernel.InvokePromptAsync(
    "You are a service that generates a random fantasy character for a novel.",
    new(new OpenAIPromptExecutionSettings { ResponseFormat = typeof(MyGreatNovelCharacter) }));

var resultObject = JsonSerializer.Deserialize<MyGreatNovelCharacter>(result.ToString());

Console.WriteLine(JsonSerializer.Serialize(resultObject, new JsonSerializerOptions { WriteIndented = true }));

As you can see, generating data into a structured format can be a snap using the Semantic Kernel framework.